In [1]:
import pandas as pd
import numpy as np 

In [2]:
rosters = pd.read_csv("/users/jordanwegner/Desktop/nba2/03_data/rosters.csv")
seasons = pd.read_csv("/users/jordanwegner/Desktop/nba2/03_data/seasons.csv")
teams = pd.read_csv("/users/jordanwegner/Desktop/nba2/03_data/teams.csv")
players = pd.read_csv("/users/jordanwegner/Desktop/nba2/03_data/players.csv")
gids = pd.read_csv("/users/jordanwegner/Desktop/nba2/03_data/game_ids.csv")
draft = pd.read_csv("/users/jordanwegner/Desktop/nba2/03_data/draft.csv")
pwins = pd.read_csv("/users/jordanwegner/Desktop/nba2/03_data/playoff_wins.csv")

In [3]:
#gids = gids[(gids['TEAM_ID'].isin([1610612737,1610612744]))&(gids['SEASON_YEAR'].isin(['2020-21','2019-20']))]
#gids = gids.sample(5,random_state=41)

In [18]:
seasons

,id,y1,y2
0,2000-01,2000,2001
1,2001-02,2001,2002
2,2002-03,2002,2003
3,2003-04,2003,2004
4,2004-05,2004,2005
5,2005-06,2005,2006
6,2006-07,2006,2007
7,2007-08,2007,2008
8,2008-09,2008,2009
9,2009-10,2009,2010


In [19]:
gids

,SEASON_YEAR,TEAM_ID,GAME_ID
0,2007-08,1610612737,G0020701222
1,2007-08,1610612764,G0020701218
2,2007-08,1610612746,G0020701226
3,2007-08,1610612759,G0020701223
4,2007-08,1610612745,G0020701226
...,...,...,...
55123,2010-11,1610612748,G0021000001
55124,2010-11,1610612757,G0021000002
55125,2010-11,1610612745,G0021000003
55126,2010-11,1610612747,G0021000003


In [20]:
gotten_or_error = []

In [21]:
import pandas as pd
import time
from nba_api.stats.endpoints import boxscorefourfactorsv2

def hit_boxscorefourfactorsv2(gs,skips,sn):
    # takes the game id data as input 
    # returns the four factors data for each player in each game
    headers = {'Host': 'stats.nba.com', 
               'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0', 
               'Accept': 'application/json, text/plain, */*', 'Accept-Language': 'en-US,en;q=0.5', 
               'Accept-Encoding': 'gzip, deflate, br', 
               'x-nba-stats-origin': 'stats', 
               'x-nba-stats-token': 'true', 
               'Connection': 'keep-alive', 
               'Referer': 'https://stats.nba.com/', 
               'Pragma': 'no-cache', 
               'Cache-Control': 'no-cache'}
    bsffs = []
    count = 0
    gs = gs[(~gs['GAME_ID'].isin(skips))&(gs['SEASON_YEAR']==sn)]
    for i in set(gs['GAME_ID']):
        print("Starting: {}".format(i))
        bsff = boxscorefourfactorsv2.BoxScoreFourFactorsV2(game_id=i[1:],headers = headers, timeout=60).get_data_frames()[0]
        bsffs.append(bsff)
        print("Finished: {}".format(i))
        count = count+1
        rem = len(set(gs['GAME_ID']))-count
        print("Remaining: {}".format(rem))
        time.sleep(.6)
    BSFF = pd.concat(bsffs)
    BSFF['GAME_ID'] = 'G'+BSFF['GAME_ID']
    return BSFF    

In [22]:
bs = hit_boxscorefourfactorsv2(gids,gotten_or_error,'2000-01')

Starting: G0020000596
Finished: G0020000596
Remaining: 1188
Starting: G0020001162
Finished: G0020001162
Remaining: 1187
Starting: G0020000951


ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=60)

In [ ]:
# use these for writing the completed part of the file to csv and then using it to generate the gotten or error option 

In [23]:
BSFF

NameError: name 'BSFF' is not defined

In [ ]:
bs

In [ ]:
bs.columns

In [ ]:
bs2 = bs.merge(gids[['GAME_ID','SEASON_YEAR']],how='left',on='GAME_ID')
bs2

In [ ]:
bs2['YEAR'] = [i[:4] for i in bs2['SEASON_YEAR']]
bs2

In [ ]:
bs2['PLAYER_TEAM_SEASON'] = bs2['PLAYER_ID'].astype(str)+'_'+bs2['TEAM_ID'].astype(str)+'_'+bs2['YEAR']
bs2

In [ ]:
bs2['ngroup'] = bs2.groupby('PLAYER_TEAM_SEASON').ngroup()

In [ ]:
bs2

In [ ]:
bs2['ngroup'].value_counts().sort_index()

In [ ]:
bs2['PLAYER_ID'].value_counts().sort_values()

In [ ]:
bs2.to_csv('temp.csv')